# Proyecto 2
Estuardo Díaz 16110

In [1]:
import numpy as np
from scipy import linalg

### Factorización $LU$ de matrices

In [2]:
def factorizar(A):
    P, L, U = linalg.lu(A)
    return P, L, U

### Resolver Ax = b utilizando LU

In [55]:
def resolver(A,b):
    P,L,U = factorizar(A)
    b = np.transpose(P).dot(b)
    y = np.zeros((len(b),1))
    for i in range(len(b)):
        suma = 0
        for j in range(i):
            suma = suma + L[i][j]*y[j]
        y[i] = (b[i] - suma)/L[i][i]
    x = np.zeros((len(b),1))
    for ii in range(len(b)):
        i = len(b)-1 - ii
        suma = 0
        for j in range(i+1,len(b)):
            suma = suma + U[i][j]*x[j]
        x[i] = (y[i] - suma)/U[i][i]
    return x

## a) Matriz mal condicionada

Definimos la **condición** de $A$ como $cond(A) := ||A||\cdot ||A^{-1}||$

In [56]:
def cond(A):
    inv_A = linalg.inv(A)
    return linalg.norm(A)*linalg.norm(inv_A)

Consideremos la matriz $H$ mal condicionada siguiente:

La matriz de Hilbert es una matriz $H$ de $n\times n$ cuyas entradas están dadas por $$H_{ij} = \frac{1}{i+j-1}$$  

In [57]:
def getHilbertMatrix(n = 5):
    H = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            H[i,j] = 1/(i+j+1)
    return H


In [58]:
H = getHilbertMatrix()
H

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.5       , 0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111]])

In [59]:
print("cond(H): ",cond(H))

cond(H):  480849.1169946838


Como la condición de $H$ es muy grande, sabemos que la matriz es mal condicionada. Por lo tanto, la factorización no es buena. Sin embargo, la factorización $PL U$ funciona bastante bien.

In [60]:
P, L, U = factorizar(H)
PL = P.dot(U)
print("Norma de (PL*U - H): ", linalg.norm(PL.dot(U) - H))
print("PL*U - H:")
PL.dot(U) - H

Norma de (PL*U - H):  1.0159548154646056
PL*U - H:


array([[ 0.        ,  0.04166667,  0.04259259,  0.03906746,  0.03528118],
       [-0.5       , -0.33333333, -0.24996914, -0.19995966, -0.16662574],
       [-0.33333333, -0.24305556, -0.19308642, -0.16040344, -0.13723442],
       [-0.25      , -0.2       , -0.16666667, -0.14285714, -0.125     ],
       [-0.2       , -0.16666667, -0.14285714, -0.12499949, -0.11111009]])

Si lo calculamos para $H$ con $n=100$ obtenemos aún resultados similares:

In [61]:
H = getHilbertMatrix(100)
print("cond(H): ",cond(H))
P, L, U = factorizar(H)
PL = P.dot(L)
print("Norma de (PL*U - H): ", linalg.norm(PL.dot(U) - H))
print("PL*U - H:")
PL.dot(U) - H

cond(H):  6.559574546502786e+19
Norma de (PL*U - H):  1.985159670420475e-16
PL*U - H:


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  1.73472348e-18, ...,
        -2.60208521e-18, -1.73472348e-18,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.73472348e-18,  8.67361738e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

## b) Matriz de insumo producto

La matriz de insumo-producto es una matriz que representa las compras y ventas de varios sectores de economía y se utilizan para predecir demanda o la producción que se requiere para satisfacer la demanda.

En este ejemplo, $A$ es la matriz de insumo producto sobre los sectores de Agricultura, Industria y Servicios. La variable $y$ representa la demanda y queremos calcular $x$, la producción bruta.
\begin{align}
x &= Ax + y \\
x - Ax &= y \\
x(I - A) &= y\\
x &= (I-A)^{-1}y
\end{align}

In [62]:
A = np.array([[0.2, 0.1, 0.2],[0.5, 0.2, 0.1],[0.3, 0.7, 0.1]])
print("A:\n",A)

A:
 [[0.2 0.1 0.2]
 [0.5 0.2 0.1]
 [0.3 0.7 0.1]]


In [63]:
y = np.array([[600],[1000],[700]])
print("y:\n",y)

y:
 [[ 600]
 [1000]
 [ 700]]


Llamamos la la matriz $(I-A)$ la matriz de Leontief y $(I-A)^{-1}$ la matriz inversa de Leontief

In [64]:
Leontief = np.identity(3)- A
print("Matriz de Leontief:\n", Leontief)
invLeontief = linalg.inv(Leontief)
print("Matriz inversa de Leontief:\n", invLeontief)

Matriz de Leontief:
 [[ 0.8 -0.1 -0.2]
 [-0.5  0.8 -0.1]
 [-0.3 -0.7  0.9]]
Matriz inversa de Leontief:
 [[1.83615819 0.64971751 0.48022599]
 [1.3559322  1.86440678 0.50847458]
 [1.66666667 1.66666667 1.66666667]]


Sin embargo para calcular la inversa, podemos utilizar la factorización $LU$ para encontrar la solucion

Encontramos $x$ como $x = (I-A)^{-1}y$

In [65]:
I = np.identity(len(y))
x = resolver(I-A,y)
print("x:\n",x)

x:
 [[2087.57062147]
 [3033.89830508]
 [3833.33333333]]


Notamos que la solución es efectivamente correcta

In [66]:
print("x - (Ax+y):")
print(x-(A.dot(x)+y))

x - (Ax+y):
[[0.00000000e+00]
 [0.00000000e+00]
 [4.54747351e-13]]


## c) Regresión múltiple

In [67]:
A = np.array([[2.0,4.0,5.0],[6.0,9.0,8.0],[4.1,5.0,3.0]])

### Referencias
* Weisstein, Eric W. "Hilbert Matrix." From MathWorld--A Wolfram Web Resource. http://mathworld.wolfram.com/HilbertMatrix.html
* Prof. Waldo Marquez González, "La Matriz de Leontief", EL PROBLEMA ECONOMICO DE LAS RELACIONES INTERINDUSTRIALES. http://www.ehu.eus/Jarriola/Docencia/EcoEsp/matriz-de-leontief.pdf